In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 
          'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 
          'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 
          'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin',
          'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 
          'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
          'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 
          'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 
          'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 
          'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 
          'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

    Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. 

In [3]:
def get_list_of_university_towns():
    #a={}
    towns = pd.DataFrame(columns=['State', 'RegionName'])
    with open('university_towns.txt','r') as file:
        for line in file:
            if '[edit]' in line:
                state = line.replace('[edit]\n', '')
                #a[state] = []
            else:
                town = re.sub(' *\(.*\n', '', line)
                #a[state].append(town) 
                df = pd.DataFrame([[state, town]], columns=['State', 'RegionName'])
                towns = towns.append(df)
    return towns

In [4]:
untow = pd.read_table('university_towns.txt', header = None, sep='[0-9]$', engine='python')

In [5]:
untow.head()

,0
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]


In [6]:
towns = pd.DataFrame(columns=['State', 'RegionName'])

In [7]:
a={}
with open('university_towns.txt','r') as file:
    for line in file:
        if '[edit]' in line:
            state = line.replace('[edit]\n', '')
            a[state] = []
        else:
            town = re.sub(' *\(.*\n', '', line)
            a[state].append(town) 
            df = pd.DataFrame([[state, town]], columns=['State', 'RegionName'])
            towns = towns.append(df)

In [8]:
#  classmethod DataFrame.from_dict(data, orient='columns', dtype=None)
#towns = pd.DataFrame.from_dict(a, orient = 'columns')
towns.head()

,State,RegionName
0,Alabama,Auburn
0,Alabama,Florence
0,Alabama,Jacksonville
0,Alabama,Livingston
0,Alabama,Montevallo


In [9]:
get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
0,Alabama,Florence
0,Alabama,Jacksonville
0,Alabama,Livingston
0,Alabama,Montevallo


Returns the year and quarter of the recession start time as a string value in a format such as 2005q3

In [10]:
def get_recession_start():
    a=[]
    data = pd.read_excel('gdplev.xlsx', skiprows=8, parse_cols=[4,5,6], header=None)
    data.columns = ['YearQ', 'GDP1', 'GDP2']
    for i in range (len(data)-3):
        if data['GDP1'].iloc[i]<data['GDP1'].iloc[i+1] and data['GDP1'].iloc[i+1]>data['GDP1'].iloc[i+2] and data['GDP1'].iloc[i+2]>data['GDP1'].iloc[i+3]:
            a.append(data['YearQ'].iloc[i+1])
    return a

In [11]:
get_recession_start()

['1948Q4', '1953Q2', '1957Q3', '2008Q3']

### Additional materials, extract the Year and Quartal with regular expression

In [12]:
data = pd.read_excel('gdplev.xlsx', skiprows=8, parse_cols=[4,5,6], header=None)

In [13]:
data.columns = ['YearQ', 'GDP1', 'GDP2']
data.head()

,YearQ,GDP1,GDP2
0,1947Q1,243.1,1934.5
1,1947Q2,246.3,1932.3
2,1947Q3,250.1,1930.3
3,1947Q4,260.3,1960.7
4,1948Q1,266.2,1989.5


In [14]:
data['Year'] = data['YearQ'].map(lambda x: int(re.findall('(.*)Q', x)[0]))
data['Quart'] = data['YearQ'].map(lambda x: int(re.findall('.*Q([1-4])', x)[0]))
data = data[['YearQ','Year', 'Quart', 'GDP1', 'GDP2']]

In [15]:
data.head()

,YearQ,Year,Quart,GDP1,GDP2
0,1947Q1,1947,1,243.1,1934.5
1,1947Q2,1947,2,246.3,1932.3
2,1947Q3,1947,3,250.1,1930.3
3,1947Q4,1947,4,260.3,1960.7
4,1948Q1,1948,1,266.2,1989.5


In [16]:
#data['r'].map(lambda x: int(re.findall('(.*)Q', x)[0]))
a=np.array(data['GDP1'])

In [17]:
b = np.array(a[0])
b = np.hstack((b, a[:-1]))
c = a-b
c[c<0] = -1
c[c>=0] = 0

In [18]:
data['delta'] = c

In [19]:
data.head()

,YearQ,Year,Quart,GDP1,GDP2,delta
0,1947Q1,1947,1,243.1,1934.5,0.0
1,1947Q2,1947,2,246.3,1932.3,0.0
2,1947Q3,1947,3,250.1,1930.3,0.0
3,1947Q4,1947,4,260.3,1960.7,0.0
4,1948Q1,1948,1,266.2,1989.5,0.0


In [20]:
for i in range (c.shape[0]-2):
    if c[i]==0 and c[i+1]==-1 and c[i+2]==-1:
        print(data['YearQ'].iloc[i+1])

1949Q1
1953Q3
1957Q4
2008Q4


In [21]:
c

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  0., -1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0., -1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0

In [22]:
data.iloc[246:250]

,YearQ,Year,Quart,GDP1,GDP2,delta
246,2008Q3,2008,3,14843.0,14891.6,0.0
247,2008Q4,2008,4,14549.9,14577.0,-1.0
248,2009Q1,2009,1,14383.9,14375.0,-1.0
249,2009Q2,2009,2,14340.4,14355.6,-1.0


## Returns the year and quarter of the recession end time as a string value in a format such as 2005q3


In [23]:
def get_recession_end():
    a=[]
    data = pd.read_excel('gdplev.xlsx', skiprows=8, parse_cols=[4,5,6], header=None)
    data.columns = ['YearQ', 'GDP1', 'GDP2']
    for i in range (len(data)-4):
        if data['GDP1'].iloc[i]>data['GDP1'].iloc[i+1] and data['GDP1'].iloc[i+1]>data['GDP1'].iloc[i+2] and data['GDP1'].iloc[i+2]<data['GDP1'].iloc[i+3] and data['GDP1'].iloc[i+3]<data['GDP1'].iloc[i+4]:
            a.append(data['YearQ'].iloc[i+4])
            print(i+4)
    return a

In [24]:
get_recession_end()

30
46
251


['1954Q3', '1958Q3', '2009Q4']

## Returns the year and quarter of the recession bottom time as a string value in a format such as 2005q3'''

In [25]:
# I am going to fing a min GDP between recession starts
def get_recession_bottom():
    # get recession start number in the list
    a=[]
    data = pd.read_excel('gdplev.xlsx', skiprows=8, parse_cols=[4,5,6], header=None)
    data.columns = ['YearQ', 'GDP1', 'GDP2']
    for i in range (len(data)-3):
        if data['GDP1'].iloc[i]<data['GDP1'].iloc[i+1] and data['GDP1'].iloc[i+1]>data['GDP1'].iloc[i+2] and data['GDP1'].iloc[i+2]>data['GDP1'].iloc[i+3]:
            a.append(i)
    a = [0]+a+[len(data)]
    # we can print the numbers of starts in the next raw (not active now)
    #print(a) 
    # find the min GDP between two recession starts
    c=[]
    for i in range (len(a)-1):
        data1 = data.iloc[a[i]:a[i+1]+1]
        c.append(data1.YearQ[data1['GDP1']==data1['GDP1'].min()].iloc[0])
    # In the answer exclude the first item since we do not know whether there was a start of recession
    return c[1:]

In [26]:
get_recession_bottom()

['1949Q4', '1954Q1', '1958Q1', '2009Q2']

In [27]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    # Replace state names
    data["State"].replace(states, inplace = True)
    data.set_index(['State','RegionName'], inplace=True, drop=True)
    # Exclude not necessary columns
    data = data.loc[:,'2000-01':'2016-09']
    # convert column index to datetime and to the format with quarter
    data.columns = data.columns.to_datetime()
    data.columns = data.columns.to_period("Q")
    a=data.columns
    data.columns = a.strftime('%FQ%q')
    # Group data by column names
    housing = data.groupby(data.columns, axis = 1).mean()
    # format a numbers with two digits after comma
    housing = housing.applymap("{0:.2f}".format)
    housing = housing.astype(float)
    
    return housing

In [28]:
convert_housing_data_to_quarters().head()

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,222048.00,231067.33,239686.00,248334.00,257128.33,266202.00,274140.33,282633.00,288250.67,282882.33,...,504578.67,514086.67,522221.00,530637.67,540241.00,550108.33,559214.67,568415.00,576374.00,584177.00
California,Los Angeles,211069.33,218402.00,225263.67,231445.33,238111.33,245102.67,252395.67,260720.67,270764.33,282815.33,...,490171.33,502080.33,509342.00,520398.67,532149.00,540818.67,552514.33,564991.67,574973.33,587002.67
Illinois,Chicago,144396.33,150046.33,155931.67,161776.67,166541.67,168864.33,173347.00,179546.00,184872.33,189735.00,...,184661.67,187220.00,190557.67,193443.33,198347.67,200033.67,202756.33,205513.00,209510.67,212945.67
Texas,Houston,106602.33,106375.67,106136.67,106816.00,106766.67,107097.33,108194.67,109738.67,110623.33,111215.33,...,126781.67,129350.67,132421.67,135431.33,139072.00,141771.33,144408.67,147913.67,151409.67,155948.00
Pennsylvania,Philadelphia,51655.00,52764.33,53640.33,54395.33,55100.33,55999.33,56512.33,57636.33,59343.67,61024.67,...,102280.00,103385.67,104821.00,105649.67,107148.00,108451.33,108780.67,109656.67,112880.33,117774.33


### Additional materials

In [29]:
data = pd.read_csv('City_Zhvi_AllHomes.csv')

In [30]:
data.shape

(12454, 270)

In [31]:
# Replace state names
data["State"].replace(states, inplace = True)

In [32]:
data.set_index(['State','RegionName'], inplace=True, drop=True)

In [33]:
data.head()

,,RegionID,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,...,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,6181,New York,Queens,1,169892.0,170502.0,170709.0,170832.0,171053.0,171145.0,...,631469,636652,640977,644589,647839,650434,651823,653102,654248,655109
California,Los Angeles,12447,Los Angeles-Long Beach-Anaheim,Los Angeles,2,158338.0,157973.0,157641.0,157425.0,157296.0,157227.0,...,626646,630765,635642,641292,646730,651828,657699,664107,669843,674140
Illinois,Chicago,17426,Chicago,Cook,3,113575.0,115946.0,117006.0,117450.0,117803.0,117753.0,...,220668,221047,221146,221485,222481,223617,224342,225227,225903,226073
Texas,Houston,39051,Houston,Harris,4,87135.0,87267.0,86778.0,86544.0,86908.0,87647.0,...,168181,168142,168404,169272,170477,171456,172291,173404,174951,176266
Pennsylvania,Philadelphia,13271,Philadelphia,Philadelphia,5,48353.0,48292.0,48255.0,48185.0,48063.0,47964.0,...,132469,133558,135184,136701,137999,139512,141134,142819,144311,145291


In [34]:
# Exclude not necessary columns
data = data.loc[:,'2000-01':'2016-09']

In [35]:
data.head()

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,219468.0,222128.0,224548.0,227588.0,231200.0,234414.0,237076.0,239615.0,242367.0,245331.0,...,561887,564500,568564,572181,574156,576322,578644,580262,583750,588519
California,Los Angeles,208745.0,211039.0,213424.0,216089.0,218356.0,220761.0,223175.0,225385.0,227231.0,229185.0,...,556374,561324,565219,568432,571710,575065,578145,581925,586742,592341
Illinois,Chicago,142568.0,144405.0,146216.0,148150.0,150047.0,151942.0,153771.0,155827.0,158197.0,160220.0,...,203656,204410,205432,206697,208273,209616,210643,211614,213095,214128
Texas,Houston,105952.0,106664.0,107191.0,106876.0,106263.0,105988.0,106020.0,106172.0,106218.0,106426.0,...,145426,146828,147989,148924,149939,151398,152892,154434,156000,157410
Pennsylvania,Philadelphia,51365.0,51657.0,51943.0,52325.0,52800.0,53168.0,53397.0,53616.0,53908.0,54160.0,...,108793,108936,109527,110507,111535,112724,114382,116167,117858,119298


In [36]:
# convert column index to datetime and to the format with quarter
data.columns = data.columns.to_datetime()
data.columns = data.columns.to_period("Q")

In [37]:
data.head()

,,2000Q1,2000Q1,2000Q1,2000Q2,2000Q2,2000Q2,2000Q3,2000Q3,2000Q3,2000Q4,...,2015Q4,2016Q1,2016Q1,2016Q1,2016Q2,2016Q2,2016Q2,2016Q3,2016Q3,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,219468.0,222128.0,224548.0,227588.0,231200.0,234414.0,237076.0,239615.0,242367.0,245331.0,...,561887,564500,568564,572181,574156,576322,578644,580262,583750,588519
California,Los Angeles,208745.0,211039.0,213424.0,216089.0,218356.0,220761.0,223175.0,225385.0,227231.0,229185.0,...,556374,561324,565219,568432,571710,575065,578145,581925,586742,592341
Illinois,Chicago,142568.0,144405.0,146216.0,148150.0,150047.0,151942.0,153771.0,155827.0,158197.0,160220.0,...,203656,204410,205432,206697,208273,209616,210643,211614,213095,214128
Texas,Houston,105952.0,106664.0,107191.0,106876.0,106263.0,105988.0,106020.0,106172.0,106218.0,106426.0,...,145426,146828,147989,148924,149939,151398,152892,154434,156000,157410
Pennsylvania,Philadelphia,51365.0,51657.0,51943.0,52325.0,52800.0,53168.0,53397.0,53616.0,53908.0,54160.0,...,108793,108936,109527,110507,111535,112724,114382,116167,117858,119298


In [38]:
a=data.columns
data.columns = a.strftime('%FQ%q')

In [39]:
# Group data by column names
housing = data.groupby(data.columns, axis = 1).mean()
# format a numbers with two digits after comma
housing = housing.applymap("{0:.2f}".format)

In [40]:
housing = housing.astype(float)

In [41]:
housing.head()

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,222048.00,231067.33,239686.00,248334.00,257128.33,266202.00,274140.33,282633.00,288250.67,282882.33,...,504578.67,514086.67,522221.00,530637.67,540241.00,550108.33,559214.67,568415.00,576374.00,584177.00
California,Los Angeles,211069.33,218402.00,225263.67,231445.33,238111.33,245102.67,252395.67,260720.67,270764.33,282815.33,...,490171.33,502080.33,509342.00,520398.67,532149.00,540818.67,552514.33,564991.67,574973.33,587002.67
Illinois,Chicago,144396.33,150046.33,155931.67,161776.67,166541.67,168864.33,173347.00,179546.00,184872.33,189735.00,...,184661.67,187220.00,190557.67,193443.33,198347.67,200033.67,202756.33,205513.00,209510.67,212945.67
Texas,Houston,106602.33,106375.67,106136.67,106816.00,106766.67,107097.33,108194.67,109738.67,110623.33,111215.33,...,126781.67,129350.67,132421.67,135431.33,139072.00,141771.33,144408.67,147913.67,151409.67,155948.00
Pennsylvania,Philadelphia,51655.00,52764.33,53640.33,54395.33,55100.33,55999.33,56512.33,57636.33,59343.67,61024.67,...,102280.00,103385.67,104821.00,105649.67,107148.00,108451.33,108780.67,109656.67,112880.33,117774.33


In [42]:
housing.shape

(12454, 67)

In [43]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    # we will use only period 2000 - nowdays 
    start = get_recession_start()[-1]
    bot = get_recession_bottom()[-1]
    housing = convert_housing_data_to_quarters()
    housing=housing.reset_index()
    housing['ratio'] = housing[start]/housing[bot]
    housing = housing.dropna()
    housing.reset_index(inplace=True)
    all_towns = housing[['State','RegionName', 'ratio']]
    univ = get_list_of_university_towns()
    univ = pd.merge(all_towns, univ, how = 'inner', on = ['State','RegionName'])
    non_univ = all_towns[~all_towns.index.isin(univ[['State','RegionName']])]
    # Testing the hypotheses
    t_stat, p_value = ttest_ind(univ["ratio"], non_univ["ratio"])
    if p_value < 0.01:
        different = True
    else:
        different = False
    if t_stat < 0:
        better = "university town"
    else:
        better = "non-university town"
    return (different, p_value, better)

In [44]:
run_ttest()

(True, 0.0013918403391123798, 'university town')

### Additional Materials

In [45]:
# we will use only period 2000 - nowdays 
start = get_recession_start()[-1]
bot = get_recession_bottom()[-1]

In [46]:
start, bot

('2008Q3', '2009Q2')

In [47]:
housing = convert_housing_data_to_quarters()

In [48]:
housing=housing.reset_index()

In [49]:
housing['ratio'] = housing[start]/housing[bot]
housing.reset_index(inplace=True)
housing = housing.dropna()

In [50]:
all_towns = housing[['State','RegionName', 'ratio']].copy()

In [51]:
univ = get_list_of_university_towns()

In [52]:
univ = pd.merge(all_towns, univ, how = 'inner', on = ['State','RegionName'])

In [53]:
non_univ = all_towns[~all_towns.index.isin(univ[['State','RegionName']])]

In [54]:
# Testing the hypotheses
t_stat, p_value = ttest_ind(univ["ratio"], non_univ["ratio"])

In [55]:
if p_value < 0.01:
    different = True
else:
    different = False
if t_stat < 0:
    better = "university town"
else:
    better = "non-university town"
print(different, p_value, better)

True 0.00139184033911 university town


In [56]:
univ.head()

,State,RegionName,ratio
0,Texas,Houston,1.026455
1,Nevada,Las Vegas,1.303172
2,California,San Diego,1.037667
3,Texas,Austin,1.024922
4,Texas,Fort Worth,1.020863
